# Demo search backend

The notebook demos basic search functionality using OpenSearch and the Haystack framework. You must have Docker Desktop installed and be a part of the [MoJ Docker org](https://user-guide.operations-engineering.service.justice.gov.uk/documentation/services/dockerhub.html#docker) (so that you're covered by a licence) prior to using OpenSearch.

To install necessary packages, run `pip install -e '.[search_backend, dev]'`.

Before running this notebook, set up an Opensearch container (see docker-compose.yml) by running:
```
docker compose up localstack
```
Or alternatively follow instructions here: https://docs.haystack.deepset.ai/v2.0/docs/opensearchbm25retriever

In [1]:
import json
from haystack import Document
from haystack_integrations.document_stores.opensearch import OpenSearchDocumentStore
from search_backend.indexing_pipeline import IndexingPipeline
from search_backend.retrieval_pipeline import RetrievalPipeline
from search_backend.search import Search


cfg = {
    # Optional arg for the OpenSearch docstore, to prevent trying to index everything in one go
    "index_batch_size": 10,
    # Select embedding model for the semantic search. This should be a sentence-similarity
    # model available on Huggingface: https://huggingface.co/models?pipeline_tag=sentence-similarity
    "dense_embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
    # The value of the embedding dimension must match that specified for the model defined above
    "embedding_dim": 384,
    # Language model used to rank search results better than the embedding retrieval can
    "rerank_model": "cross-encoder/ms-marco-MiniLM-L-2-v2",
}

/Users/chloe.pugh/rd-search-backend/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get some text data

This dataset is based on Wikipedia introductions to the Seven Wonders of the Ancient World.

In [2]:
with open('../tests/data/demo_data.json') as f:
    doc_list = json.load(f)

In [ ]:
doc_list

In [ ]:
# Put into Haystack Document instances
docs = [Document(**content) for content in doc_list]

## Set up Opensearch

In [2]:
# Connect to an existing Opensearch container - see docker-compose.yml for Opensearch settings
query_document_store = OpenSearchDocumentStore(
    hosts="http://0.0.0.0:4566/opensearch/eu-west-2/rd-demo",
    use_ssl=False,
    verify_certs=False,
    http_auth=("localstack", "localstack"),
    embedding_dim=cfg["embedding_dim"],
    batch_size=cfg["index_batch_size"],
)

In [ ]:
# Write the documents to the vector store
indexer = IndexingPipeline(query_document_store, dense_embedding_model=cfg["dense_embedding_model"], semantic=True)
indexer.index_docs(docs)

## Run BM25 search

In [8]:
bm25_pipeline = RetrievalPipeline(query_document_store).setup_bm25_pipeline()
bm25_search_init = Search(bm25_pipeline)

In [9]:
test_query = "lighthouse"
# test_query = "wonder that features plants"
results = bm25_search_init.bm25_search(test_query, top_k=3)

for doc in results:
    print(f'{doc.meta["title"]} - Score: {doc.score}')
    print(doc.content)

Great Pyramid of Giza - Score: 0.6114707644356653
The Great Pyramid of Giza is the largest Egyptian pyramid. It served as the tomb of pharaoh Khufu, who ruled during the Fourth Dynasty of the Old Kingdom. Built c. 2600 BC, over a period of about 26 years, the pyramid is the oldest of the Seven Wonders of the Ancient World, and the only wonder that has remained largely intact. It is the most 
Hanging Gardens of Babylon - Score: 0.5952300196552911
to have been built in the ancient city of Babylon, near present-day Hillah, Babil province, in Iraq. The Hanging Gardens' name is derived from the Greek word κρεμαστός (kremastós, lit. 'overhanging'), which has a broader meaning than the modern English word 'hanging' and refers to trees being planted on a raised structure such as a terrace.
Colossus of Rhodes - Score: 0.5276781669100258
Wonders of the Ancient World, it was constructed to celebrate the successful defence of Rhodes city against an attack by Demetrius I of Macedon, who had besiege

In [ ]:
results[0].meta

In [ ]:
results

## Run semantic search

In [6]:
semantic_pipeline = RetrievalPipeline(
    query_document_store,
    dense_embedding_model=cfg['dense_embedding_model'],
    rerank_model=cfg['rerank_model']
).setup_semantic_pipeline()
semantic_search_init = Search(semantic_pipeline)

In [7]:
test_query = "wonder that features plants"
results = semantic_search_init.semantic_search(test_query, top_k=3, threshold=0.00001)

for doc in results:
    print(f'{doc.meta["title"]} - Score: {doc.score}')
    print(doc.content)

Running search...


Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00, 49.36it/s]

Hanging Gardens of Babylon - Score: 3.407466283533722e-05
The Hanging Gardens of Babylon were one of the Seven Wonders of the Ancient World listed by Hellenic culture. They were described as a remarkable feat of engineering with an ascending series of tiered gardens containing a wide variety of trees, shrubs, and vines, resembling a large green mountain constructed of mud bricks. It was said to have been built in the ancient city 


## Hybrid search

In [3]:
hybrid_pipeline = RetrievalPipeline(
    query_document_store,
    dense_embedding_model=cfg['dense_embedding_model'],
    rerank_model=cfg['rerank_model']
).setup_hybrid_pipeline()
hybrid_search_init = Search(hybrid_pipeline)

In [5]:
test_query = "wonder that features plants"
results = hybrid_search_init.hybrid_search(test_query, bm25_top_k=3, semantic_top_k=3, threshold=0.000001)

for doc in results:
    print(f'{doc.meta["title"]} - Score: {doc.score}')
    print(doc.content)

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00, 54.12it/s]


Hanging Gardens of Babylon - Score: 0.9838709677419354
to have been built in the ancient city of Babylon, near present-day Hillah, Babil province, in Iraq. The Hanging Gardens' name is derived from the Greek word κρεμαστός (kremastós, lit. 'overhanging'), which has a broader meaning than the modern English word 'hanging' and refers to trees being planted on a raised structure such as a terrace.
Hanging Gardens of Babylon - Score: 0.9692307692307693
The Hanging Gardens of Babylon were one of the Seven Wonders of the Ancient World listed by Hellenic culture. They were described as a remarkable feat of engineering with an ascending series of tiered gardens containing a wide variety of trees, shrubs, and vines, resembling a large green mountain constructed of mud bricks. It was said to have been built in the ancient city 
Great Pyramid of Giza - Score: 0.5
The Great Pyramid of Giza is the largest Egyptian pyramid. It served as the tomb of pharaoh Khufu, who ruled during the Fourth Dynasty 